In [ ]:
from pathlib import Path

SYFT_FLWR_PROJECT_PATH = Path("./fl-diabetes-prediction")
SYFTBOX_DATASET_NAME = "pima-indians-diabetes-database"
RUN_SIMULATION = True

assert SYFT_FLWR_PROJECT_PATH.exists()

## Log into the data owners' datasites

In [ ]:
import syft_rds as sy
from syft_core import Client

DS = Client.load().email
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"
DO3 = "do3@openmined.org"
DO4 = "do4@openmined.org"
DO5 = "do5@openmined.org"

do_client_1 = sy.init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = sy.init_session(host=DO2)
print("Logged into: ", do_client_2.host)

do_client_3 = sy.init_session(host=DO3)
print("Logged into: ", do_client_3.host)

do_client_4 = sy.init_session(host=DO4)
print("Logged into: ", do_client_4.host)

do_client_5 = sy.init_session(host=DO5)
print("Logged into: ", do_client_5.host)

## Explore the datasets

In [ ]:
do_clients = [do_client_1, do_client_2, do_client_3, do_client_4, do_client_5]
mock_paths = []
for client in do_clients:
    dataset = client.dataset.get(name=SYFTBOX_DATASET_NAME)
    mock_paths.append(dataset.get_mock_path())
    print(dataset)

## Bootstrapping and run `syft_flwr` simulation

In [ ]:
import syft_flwr

syft_flwr.bootstrap(SYFT_FLWR_PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2, DO3, DO4, DO5])

## Run `flwr` and `syft_flwr` simulations (optional)

In [ ]:
if RUN_SIMULATION:
    !flwr run {SYFT_FLWR_PROJECT_PATH}

In [ ]:
if RUN_SIMULATION:
    syft_flwr.run(SYFT_FLWR_PROJECT_PATH, mock_paths)

## Submit jobs

Remove `__pycache__` (if exists) from running the simulation

In [ ]:
!rm -rf SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"

In [ ]:
for client in do_clients:
    job = client.jobs.submit(
        name="Syft Flower Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=SYFT_FLWR_PROJECT_PATH,
        dataset_name=SYFTBOX_DATASET_NAME,
        tags=["federated learning", "fl", "syft_flwr", "flwr"],
        entrypoint="main.py",
    )
    print(job)

## DS starts the FL server code

In [ ]:
import os

os.environ["LOGURU_LEVEL"] = "DEBUG"
!uv run {str(SYFT_FLWR_PROJECT_PATH / "main.py")} --active